In [76]:
import random
    
QUERY_POOL = list(range(11))
random.shuffle(QUERY_POOL)
for index in range(4):
    # get the last 'batch_size_test' nodes as query nodes
    query_indices = QUERY_POOL[:3]
    QUERY_POOL = QUERY_POOL[3:]
    print(index,'query_indices', query_indices)

0 query_indices [7, 3, 2]
1 query_indices [8, 5, 6]
2 query_indices [9, 4, 0]
3 query_indices [1, 10]


In [77]:
import pandas as pd
# baic transformer Decoder model
import torch
import torch.nn as nn
import torch.nn.functional as Fun
import torch.optim as optim
import numpy as np
import xformers.ops as xops
import math 
from typing import Optional, Union
from torch import Tensor
import random

main_df = pd.read_csv('../data/RAW_data/adult.csv')
main_df.head()
DEVICE = 'cuda'
# DEVICE = 'cpu'
torch.random.manual_seed(42)
random.seed(42)
np.random.seed(42)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms(True)

## 5 FOLD (transformed input)

In [1]:
import numpy as np
from sklearn.model_selection import KFold
import xgboost as xgb
from sklearn.metrics import roc_auc_score, accuracy_score
# import labelencoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from utils.utils import *
import pandas as pd

kf = KFold(n_splits=5, shuffle=True,random_state=42)
AUCS = []
ACCS = []
# main_df = pd.read_csv('../data/RAW_data/adult.csv')
main_df = pd.read_csv(select_dataset('compas'))
# main_df = select_dataset('adult')
# main_df =X_trans

# 進行5-fold交叉驗證
for index, (train_index, test_index) in enumerate(kf.split(main_df)):
    X_train, X_test = main_df.loc[train_index], main_df.loc[test_index]
    NUM, CAT, TARGET = get_feilds_attributes()
    for column in NUM:
        # min-max normalization
        #X_train[column] = (X_train[column] - X_train[column].mean()) / (X_train[column].std())
        #X_test[column] = (X_test[column] - X_test[column].mean()) / (X_test[column].std())
        X_train[column] = X_train[column].astype(np.float32)
        pass
    le = LabelEncoder()
    Y_train, Y_test = X_train[TARGET] , X_test[TARGET]
    Y_train = le.fit_transform(Y_train)
    Y_test = le.fit_transform(Y_test)

    
    X_train = X_train.drop(columns=TARGET)
    X_test = X_test.drop(columns=TARGET)
    
    for column in CAT:
        if column == TARGET:
            continue
        le = LabelEncoder()
        le.fit(main_df[column].astype(str))
        X_train[column] = le.transform(X_train[column].astype(str))
        X_test[column] = le.transform(X_test[column].astype(str))
    
    print(f'Fold {index}:')
    print(f'X_train: {X_train.shape}, X_test: {X_test.shape}')
    print(f'Y_train: {Y_train.shape}, Y_test: {Y_test.shape}')
    # print(X_train.head())
    
    from sklearn.ensemble import RandomForestClassifier
    
    model = RandomForestClassifier(random_state=42, n_jobs=-1, n_estimators=2000)
    # model = xgb.XGBClassifier(random_state=42, n_jobs=-1, n_estimators=100)
    model.fit(X_train, Y_train)
    Y_prob = model.predict_proba(X_test)  #预测概率
    Y_pred = model.predict(X_test) 
    acc = accuracy_score(Y_test, Y_pred)
    auc = roc_auc_score(Y_test, Y_prob[:, 1])
    AUCS.append(auc)
    ACCS.append(acc)
    print(f'auc: {auc}')
    print(f'acc: {acc}')
    
print(f'Average AUC: {np.mean(AUCS)}')
print(f'Average ACC: {np.mean(ACCS)}')

=================[dataset is set to compas ]=================
[0 1]
Fold 0:
X_train: (3972, 11), X_test: (994, 11)
Y_train: (3972,), Y_test: (994,)
auc: 0.6586965147539922
acc: 0.596579476861167
[0 1]
Fold 1:
X_train: (3973, 11), X_test: (993, 11)
Y_train: (3973,), Y_test: (993,)
auc: 0.6595726947471133
acc: 0.608257804632427
[0 1]
Fold 2:
X_train: (3973, 11), X_test: (993, 11)
Y_train: (3973,), Y_test: (993,)
auc: 0.6392174999594308
acc: 0.6032225579053374
[0 1]
Fold 3:
X_train: (3973, 11), X_test: (993, 11)
Y_train: (3973,), Y_test: (993,)
auc: 0.6923293795028234
acc: 0.6475327291037261
[0 1]
Fold 4:
X_train: (3973, 11), X_test: (993, 11)
Y_train: (3973,), Y_test: (993,)
auc: 0.6734875589526752
acc: 0.6314199395770392
Average AUC: 0.664660729583207
Average ACC: 0.6174025016159395
